In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("archive/winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
input_cols = list(df.columns)[:-1]
input_cols

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [4]:
output_cols = ['quality']

In [5]:
def dataframe_to_arrays(df):
    # Make a copy of the original dataframe
    df1 = df.copy(deep=True)
    # Extract input & outupts as numpy arrays
    inputs_array = df1[input_cols].to_numpy()
    targets_array = df1[output_cols].to_numpy()
    return inputs_array, targets_array

In [6]:
inputs_array, targets_array = dataframe_to_arrays(df)
inputs_array, targets_array

(array([[ 7.4  ,  0.7  ,  0.   , ...,  3.51 ,  0.56 ,  9.4  ],
        [ 7.8  ,  0.88 ,  0.   , ...,  3.2  ,  0.68 ,  9.8  ],
        [ 7.8  ,  0.76 ,  0.04 , ...,  3.26 ,  0.65 ,  9.8  ],
        ...,
        [ 6.3  ,  0.51 ,  0.13 , ...,  3.42 ,  0.75 , 11.   ],
        [ 5.9  ,  0.645,  0.12 , ...,  3.57 ,  0.71 , 10.2  ],
        [ 6.   ,  0.31 ,  0.47 , ...,  3.39 ,  0.66 , 11.   ]]),
 array([[5],
        [5],
        [5],
        ...,
        [6],
        [5],
        [6]]))

In [7]:
import torch
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

In [8]:
from torch.utils.data import DataLoader, TensorDataset, random_split
dataset = TensorDataset(inputs, targets)

In [9]:
num_rows = len(df)
val_percent = 0.8 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_df, val_df = random_split(dataset, [train_size, val_size]) 

In [10]:
batch_size = 50

In [11]:
train_loader = DataLoader(train_df, batch_size, shuffle=True)
val_loader = DataLoader(val_df, batch_size)

In [12]:
input_size = len(input_cols)
output_size = len(output_cols)

In [13]:
import torch.nn as nn
import torch.nn.functional as F
class WineModel(nn.Module):
    def __init__(self,number_of_neurons):
        super().__init__()     
        self.fc1 = nn.Linear(input_size, number_of_neurons)
        self.fc2 = nn.Linear(number_of_neurons,output_size)
        # fill this (hint: use input_size & output_size defined above)
        #model initialized with random weight
        
    def forward(self, xb):
        xb = F.relu(self.fc1(xb))
        out = self.fc2(xb)             # batch wise forwarding
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)         
        # Calcuate loss
        loss = F.l1_loss(out, targets)  # batch wise training step and loss
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss =F.l1_loss(out, targets)       # batch wise validation and loss    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine val losses of all batches as average
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 500 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [14]:
model =  WineModel(4)

In [15]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2488, -0.1014, -0.1495,  0.1991,  0.2145, -0.0309, -0.2576, -0.2291,
           0.1672,  0.2773, -0.2660],
         [ 0.1557,  0.2515, -0.0027, -0.0020,  0.0298, -0.0991,  0.0755,  0.2055,
           0.0665,  0.2009,  0.1248],
         [-0.1697, -0.1173,  0.1503, -0.0264, -0.1701, -0.0901, -0.1408,  0.2201,
          -0.0386,  0.1766,  0.2703],
         [ 0.2657,  0.2363, -0.3007,  0.2345, -0.2927, -0.0550, -0.2398, -0.2045,
           0.1676, -0.0067, -0.0695]], requires_grad=True),
 Parameter containing:
 tensor([-0.0101, -0.0851,  0.0406, -0.1580], requires_grad=True),
 Parameter containing:
 tensor([[-0.0538,  0.2672,  0.2372, -0.3570]], requires_grad=True),
 Parameter containing:
 tensor([0.3916], requires_grad=True)]

In [16]:
def evaluate2(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)  #appends total validation loss of whole validation set epoch wise
    return history

In [17]:
result = evaluate2(model,val_loader) # Use the the evaluate function
print(result)

{'val_loss': 3.881232976913452}


In [18]:
val_loss = evaluate2(model,val_loader)
val_loss

{'val_loss': 3.881232976913452}

In [19]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0) 
    predictions = model(inputs)
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [20]:
input, target = val_df[0]
predict_single(input, target, model)

Input: tensor([ 9.5000,  0.8600,  0.2600,  1.9000,  0.0790, 13.0000, 28.0000,  0.9971,
         3.2500,  0.6200, 10.0000])
Target: tensor([5.])
Prediction: tensor([1.5211])


In [21]:
class Raindrop:
    def __init__(self,x0):
        self.position = x0
    
    def evaluate(self):
    	#self.err_i=costFunc(self.position_i)
        #net = Net(int(round(self.position_i[0])))
        net = WineModel(round(self.position))
    	#print(net)
        #optimizer = optim.Adam(net.parameters(), lr=0.05)
        #def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
        #history = []
        lr = 1e-2
        epochs = 100
        optimizer = torch.optim.SGD(net.parameters(), lr)
        for epoch in range(epochs):
            # Training Phase 
            for batch in train_loader:
                loss = net.training_step(batch)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
            # Validation phase
            result = evaluate2(net, val_loader)
            net.epoch_end(epoch, result, epochs)
            #history.append(result)  #appends total validation loss of whole validation set epoch wise
        print(result['val_loss'])
        return result['val_loss']
    

In [22]:
import random
from operator import itemgetter 
class WaterCycle:
    def __init__(self,bounds,Npop,dmax, max_it):
        global num_dimensions
        
        C = 2
        
        Nsr = 5
        Nrivers = Nsr - 1
        Nstreams = Npop-Nsr
        
        num_dimesnions = 1
        
        population = []
        for i in range(Npop):
            x0 = random.randint(bounds[0],bounds[1])
            print('Raindrop ',i,' is ', x0)
            population.append(Raindrop(x0))
            
        ListOfRaindrops = []
        print('Iteration 0')
        print('************************')
        for j in range(Npop):
            print('Raindrop ',j)
            err_j = population[j].evaluate()
            ListOfRaindrops.append([j,err_j])
        
        
        res = list(sorted(ListOfRaindrops, key = itemgetter(1), reverse = False)[:Nsr]) 
        print('ListOfRaindrops ', ListOfRaindrops)
        print('res' ,res)
        #print(res[0][0])
        
        totalCostOfNsr = 0
        for i in range(Nsr):
            totalCostOfNsr = totalCostOfNsr + res[i][1]
        
        NSn = []
        for i in range(Nsr):
            NSn.append([res[i][0],round((res[i][1]/totalCostOfNsr)*Nstreams)])
            print('NSn: ',NSn)
        
        NsrList = []
        for i in NSn:
            NsrList.append(i[0])
        print('NsrList: ',NsrList)


        NstreamsList = []
        for i in ListOfRaindrops:
            if i[0] not in NsrList:
                NstreamsList.append([i[0],i[1]])
        print('NstreamsList Unsorted: ', NstreamsList)
        NstreamsList = list(sorted(NstreamsList, key = itemgetter(1), reverse = False))
        print('NstreamsList Sorted: ',NstreamsList)
        
        streams = []
        for i in NstreamsList:
            streams.append(i[0])
        print('streams',streams)
        
        i = 1
        while(i < max_it):
            print('Iteration ',i)
            print('************************')
            
            counter = 0
            limit = 0
            for p in NSn:
                prevLimit = limit
                limit = limit + p[1]
                for j in range(prevLimit,limit):
                    if j <= len(NstreamsList):
                        drop = NstreamsList[j][0]
                        print('Raindrop ',drop)
                        population[drop].position = population[drop].position + (random.uniform(0,1) * C * (population[p[0]].position - population[drop].position))
                        if population[drop].position > bounds[1]:
                            population[drop].position = bounds[1]
                        elif population[drop].position < bounds[0]:
                            population[drop].position = bounds[0]
                        if population[drop].evaluate() >  population[p[0]].evaluate():   #stream <--> river<-->sea
                            print('Interchanging stream and river!!!!!')
                            NstreamsList[j][0] = p[0]       
                            res[counter][0] = drop
                            NSn[counter][0] = drop
                        
                counter+=1
            
            
            NsrList = []
            for c in NSn:
                NsrList.append(c[0])

            print('NsrList: ',NsrList)


            NstreamsList = list(sorted(NstreamsList, key = itemgetter(1), reverse = False))
            print('NstreamsList: ',NstreamsList)

            streams = []
            for k in NstreamsList:
                streams.append(k[0])
            print('streams',streams)

            for k in range(Npop):
                print('Raindrop ',k,' is ', population[k].position)
            
            
            for j in range(1,len(NSn)):
                river = NSn[j][0]
                sea = NSn[0][0]
                print('Raindrop ', river, ' which is a river ')
                population[river].position = population[river].position + (random.uniform(0,1) * C * (population[sea].position - population[river].position))
                if population[river].position > bounds[1]:
                    population[river].position = bounds[1]
                elif population[river].position < bounds[0]:
                    population[river].position = bounds[0]
                if population[river].evaluate() > population[sea].evaluate():
                    print('Interchanging river sand sea!!!!')
                    NSn[0][0] = river
                    NSn[j][0] = sea
                
            for j in range(1,len(NSn)):
                seaPos = NSn[0][1]
                riverPos = NSn[j][1]
                if abs(seaPos - riverPos) < dmax or random.uniform(0,1) < 0.1:
                    print('Evapouration Process for river!!')
                    population[NSn[j][0]].position = bounds[0] + (random.uniform(0,1)*(bounds[1]-bounds[0]))
            
                    
            dmax = dmax - (dmax/max_it)
            NsrList = []
            for g in NSn:
                NsrList.append(g[0])

            print('NsrList: ',NsrList)


            NstreamsList = list(sorted(NstreamsList, key = itemgetter(1), reverse = False))
            print('NstreamsList: ',NstreamsList)

            streams = []
            for t in NstreamsList:
                streams.append(t[0])
            print('streams',streams)
            
            
            i = i + 1
            
        print('Final')
        print(NSn[0][1])
#--- RUN ----------------------------------------------------------------------+

    #initial = 5               # initial starting location [x1,x2...]
bounds=[1,10]  # input bounds [(x1_min,x1_max),(x2_min,x2_max)...]
WaterCycle(bounds,Npop = 17,dmax = 0.01, max_it=2)
    

Raindrop  0  is  6
Raindrop  1  is  4
Raindrop  2  is  3
Raindrop  3  is  8
Raindrop  4  is  10
Raindrop  5  is  7
Raindrop  6  is  8
Raindrop  7  is  2
Raindrop  8  is  1
Raindrop  9  is  9
Raindrop  10  is  4
Raindrop  11  is  7
Raindrop  12  is  3
Raindrop  13  is  3
Raindrop  14  is  10
Raindrop  15  is  4
Raindrop  16  is  1
Iteration 0
************************
Raindrop  0
Epoch [100], val_loss: 0.7906
0.7905924916267395
Raindrop  1
Epoch [100], val_loss: 0.7325
0.732451856136322
Raindrop  2
Epoch [100], val_loss: 1.7269
1.7269203662872314
Raindrop  3
Epoch [100], val_loss: 0.5987
0.5986627340316772
Raindrop  4
Epoch [100], val_loss: 1.2430
1.2429975271224976
Raindrop  5
Epoch [100], val_loss: 0.7256
0.7255859375
Raindrop  6
Epoch [100], val_loss: 0.7385
0.7384992837905884
Raindrop  7
Epoch [100], val_loss: 1.7449
1.74489426612854
Raindrop  8
Epoch [100], val_loss: 2.1073
2.1073148250579834
Raindrop  9
Epoch [100], val_loss: 2.5103
2.5103182792663574
Raindrop  10
Epoch [100], val_